In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder.appName("MyFirstCSVLoad").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/11 16:29:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/11 16:29:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [21]:
rating_path = "/home/nghiaht7/data-engineer/big-data-platforms-f20.mooc.fi/Mastering-Big-Data-Analytics-with-PySpark/data-sets/ml-latest-small/ratings.csv" 

ratings = spark.read.csv(
    path=rating_path,
    sep=",",
    header=True,
    quote='"',
    schema="userId INT, movieId INT, rating DOUBLE, timestamp INT",
).withColumn("timestamp", f.to_timestamp(f.from_unixtime("timestamp")))

In [22]:
movies_path = "/home/nghiaht7/data-engineer/big-data-platforms-f20.mooc.fi/Mastering-Big-Data-Analytics-with-PySpark/data-sets/ml-latest-small/movies.csv"


movies = spark.read.csv(
    path=movies_path,
    sep=",",
    header=True,
    quote='"',
    schema="movieId INT, title STRING, genres STRING",
)
movies.show(15, truncate=False)
movies.printSchema()

+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
|6      |Heat (1995)                       |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                    |Comedy|Romance                             |
|8      |Tom and Huck (1995)               |Adventure|Children                         |
|9      |Sudden Death

In [12]:
movies.where(f.col("genres") == "Action").show(5, False)
movies.where("genres == 'Action'").show(5, False)

movies.createOrReplaceTempView("movies")
spark.sql("select * from movies where genres == 'Action'").show(5, False)

+-------+-----------------------------------------------------------+------+
|movieId|title                                                      |genres|
+-------+-----------------------------------------------------------+------+
|9      |Sudden Death (1995)                                        |Action|
|71     |Fair Game (1995)                                           |Action|
|204    |Under Siege 2: Dark Territory (1995)                       |Action|
|251    |Hunted, The (1995)                                         |Action|
|667    |Bloodsport 2 (a.k.a. Bloodsport II: The Next Kumite) (1996)|Action|
+-------+-----------------------------------------------------------+------+
only showing top 5 rows

+-------+-----------------------------------------------------------+------+
|movieId|title                                                      |genres|
+-------+-----------------------------------------------------------+------+
|9      |Sudden Death (1995)                       

In [13]:
movie_genre = (
    movies.withColumn("genres_array", f.split("genres", "\|"))
    .withColumn("genre", f.explode("genres_array"))
    .select("movieId", "title", "genre")
)

In [15]:
movie_genre.show(20, False)

+-------+----------------------------------+---------+
|movieId|title                             |genre    |
+-------+----------------------------------+---------+
|1      |Toy Story (1995)                  |Adventure|
|1      |Toy Story (1995)                  |Animation|
|1      |Toy Story (1995)                  |Children |
|1      |Toy Story (1995)                  |Comedy   |
|1      |Toy Story (1995)                  |Fantasy  |
|2      |Jumanji (1995)                    |Adventure|
|2      |Jumanji (1995)                    |Children |
|2      |Jumanji (1995)                    |Fantasy  |
|3      |Grumpier Old Men (1995)           |Comedy   |
|3      |Grumpier Old Men (1995)           |Romance  |
|4      |Waiting to Exhale (1995)          |Comedy   |
|4      |Waiting to Exhale (1995)          |Drama    |
|4      |Waiting to Exhale (1995)          |Romance  |
|5      |Father of the Bride Part II (1995)|Comedy   |
|6      |Heat (1995)                       |Action   |
|6      |H

In [16]:
available_genres = movie_genre.select("genre").distinct()

In [17]:
available_genres.show()

+------------------+
|             genre|
+------------------+
|         Animation|
|          Children|
|            Comedy|
|           Fantasy|
|           Romance|
|             Crime|
|          Thriller|
|            Horror|
|           Mystery|
|               War|
|(no genres listed)|
|         Adventure|
|             Drama|
|            Action|
|            Sci-Fi|
|           Musical|
|       Documentary|
|              IMAX|
|           Western|
|         Film-Noir|
+------------------+



In [19]:
movies_without_genre = movies.where(f.col("genres") == "(no genres listed)")

In [20]:
print(movies_without_genre.count())
movies_without_genre.show()

34
+-------+--------------------+------------------+
|movieId|               title|            genres|
+-------+--------------------+------------------+
| 114335|   La cravate (1957)|(no genres listed)|
| 122888|      Ben-hur (2016)|(no genres listed)|
| 122896|Pirates of the Ca...|(no genres listed)|
| 129250|   Superfast! (2015)|(no genres listed)|
| 132084| Let It Be Me (1995)|(no genres listed)|
| 134861|Trevor Noah: Afri...|(no genres listed)|
| 141131|    Guardians (2016)|(no genres listed)|
| 141866|   Green Room (2015)|(no genres listed)|
| 142456|The Brand New Tes...|(no genres listed)|
| 143410|          Hyena Road|(no genres listed)|
| 147250|The Adventures of...|(no genres listed)|
| 149330|A Cosmic Christma...|(no genres listed)|
| 152037|  Grease Live (2016)|(no genres listed)|
| 155589|Noin 7 veljestä (...|(no genres listed)|
| 156605|            Paterson|(no genres listed)|
| 159161|Ali Wong: Baby Co...|(no genres listed)|
| 159779|A Midsummer Night...|(no genres listed